<a href="https://colab.research.google.com/github/FreeOfConfines/ExampleNNWithKerasAndTensorflow/blob/master/K_Nearest_Neighbor_Classification_with_Tensorflow_on_Fashion_MNIST_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K-Nearest Neighbor Classification with Python/Numpy/Tensorflow on Fashion MNIST Dataset

>[K-Nearest Neighbor Classification with Python/Numpy/Tensorflow on Fashion MNIST Dataset](#updateTitle=true&folderId=1PIZOzXSYokZJyrV92mRbYGRQMq-w7swD&scrollTo=jVWQHXuc1tvK)

>>[So Far](#updateTitle=true&folderId=1PIZOzXSYokZJyrV92mRbYGRQMq-w7swD&scrollTo=QNWAWfSm2jLD)

>>[What is Fashion MNIST Dataset?](#updateTitle=true&folderId=1PIZOzXSYokZJyrV92mRbYGRQMq-w7swD&scrollTo=nz2S4Mq_4T6P)

>>[K-Nearest Neighbor Algorithm](#updateTitle=true&folderId=1PIZOzXSYokZJyrV92mRbYGRQMq-w7swD&scrollTo=uKUPyeHCfQU3)

>>>[Closeness Metric](#updateTitle=true&folderId=1PIZOzXSYokZJyrV92mRbYGRQMq-w7swD&scrollTo=uKUPyeHCfQU3)

>>>[Finding $k$ Closest Vectors in Train Set](#updateTitle=true&folderId=1PIZOzXSYokZJyrV92mRbYGRQMq-w7swD&scrollTo=uKUPyeHCfQU3)

>>>[Optimizing Parameter $k$](#updateTitle=true&folderId=1PIZOzXSYokZJyrV92mRbYGRQMq-w7swD&scrollTo=uKUPyeHCfQU3)

>>[Classification using Euclidean Distance Metric](#updateTitle=true&folderId=1PIZOzXSYokZJyrV92mRbYGRQMq-w7swD&scrollTo=g4TFt0SCdek4)

>>>[Complexity](#updateTitle=true&folderId=1PIZOzXSYokZJyrV92mRbYGRQMq-w7swD&scrollTo=g4TFt0SCdek4)

>>[Classification using L0 Distance](#updateTitle=true&folderId=1PIZOzXSYokZJyrV92mRbYGRQMq-w7swD&scrollTo=zWzeuu_0UPEE)

>>>[Complexity](#updateTitle=true&folderId=1PIZOzXSYokZJyrV92mRbYGRQMq-w7swD&scrollTo=zWzeuu_0UPEE)

>>>[Python-like Implementation](#updateTitle=true&folderId=1PIZOzXSYokZJyrV92mRbYGRQMq-w7swD&scrollTo=TfFce2zpJFuq)

>>>[Tensorflow-like Implementation](#updateTitle=true&folderId=1PIZOzXSYokZJyrV92mRbYGRQMq-w7swD&scrollTo=VwhfUWFnJasE)



##So Far

In Part-1, we had designed, trained and tested a back-propogation network on Fashion MNIST dataset. Using a two layer backprop network designed using Keras and Tensorflow, we achieved a classification accuracy of 87.2%. In this article, we will revisit the classification (or labeling) problem on this dataset but apply a classification algorithm called the K-Nearest Neighbor (or KNN). We will show that KNN achieves classification accuracy only a little worse than the backprop network.

## What is Fashion MNIST Dataset?

Fashion MNIST is a dataset of images that is given one of 10 unique labels like Tops, Trousers, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, and Ankle Boot. The dataset is divided into two groups: Training Set and Test Set; there are 60000 images in Training Set and 10000 images in Test set. Each image is a $28 \times 28$ array with values from 0 to 255.

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Download Fashion MNIST dataset
fashion_mnist = keras.datasets.fashion_mnist
(trImages, trLabels), (tImages, tLabels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
print("--------------------------")
print("Dimensions of Train Set")
print("Dimension(trImages)=",np.shape(trImages))
print("There are", np.shape(trImages)[0], "images where each image is", np.shape(trImages)[1:], "in size")
print("There are", np.shape(np.unique(tLabels))[0], "unique image labels")
print("--------------------------")
print("Dimensions of Test Set")
print("Dimension(tImages)=",np.shape(tImages), "Dimension(tLabels)=", np.shape(tLabels)[0])
print("--------------------------")

--------------------------
Dimensions of Train Set
Dimension(trImages)= (60000, 28, 28)
There are 60000 images where each image is (28, 28) in size
There are 10 unique image labels
--------------------------
Dimensions of Test Set
Dimension(tImages)= (10000, 28, 28) Dimension(tLabels)= 10000
--------------------------


##K-Nearest Neighbor Algorithm

K-Nearest Neighbor (or KNN) algorithm is a non-parametric classification algorithm. Backprop Neural Network from Part-1 is a parametric model parametrized by weights and bias values. Non-parametric model, contrary to the name, has a very large number of parameters. In the case of Fashion MNIST example, we will use the entire Train Set as parameters of KNN. 

The basic idea behind KNN is simple. Given a (test) vector or image to classify or label, find $k$ vectors or images in Train Set that are "closest" to the (test) vector or image. With the $k$ closest vectors or images, there are $k$ labels. Assign the most frequent label of $k$ labels to the (test) vector or image.

###Closeness Metric
The idea of "closest" or "closeness" depends on the metric we choose to use; for instance
* **Euclidean Distance** between two vectors $x= <x_1, x_2, x_3>$ and $y= <y_1, y_2, y_3>$ is defined as $d_{ED}:=\{(x_1-y_1)^2+(x_2-y_2)^2+(x_3-y_3)^2\}^{\frac{1}{2}}$. In academic literature, you may see this being called L1 norm of $x-y$.
* **L1 Distance** between two vectors  $x= <x_1, x_2, x_3>$ and $y= <y_1, y_2, y_3>$ is defined as $d_{L1}:=|x_1-y_1|+|x_2-y_2|+|x_3-y_3|$
* **L0 Distance** between two vectors  $x= <x_1, x_2, x_3>$ and $y= <y_1, y_2, y_3>$ is defined as the number of non-zero elements in $x-y$.

In this article, we will use the Euclidean distance and L0 distance. 

###Finding $k$ Closest Vectors in Train Set
Given a vector (or image) from Test Set, we can't say which ones in the Train Set are closest without computing the metric over all elements in the Train Set. In the case of Fashion MNIST, we compute "closeness" metric of the vector from Test Set to every element, i.e., 60000 of them, in the Train Set and this will result in 60000 distance values. As you can imagine, if the Train Set is larger then it gets all that more time-consuming or computationally consuming to find all these distance values.

###Optimizing Parameter $k$
I don't know if there is a systematic way to go about optimizing this parameter but try different "good" values for $k$ and pick the one that works best. Let's review some extreme choices for $k$:
* If $k=1$, then labeling of the test vector or image is determined by one element in the Train Set
* If $k=60000$, then label of the test vector is determined by all elements in the Train Set and if there is class imbalance, i.e., there are more images with a certain label in the Test Set, then every test vector will get the exact same label. 

## Classification using Euclidean Distance Metric

Here we use Euclidean distance metric to determine closeness of Train images to a given Test image. In the code snippet below, there are two for-loops, one looping over Test Images and another looping over Train Images. I have set parameter $k=11$; try experimenting with this parameter.

The classification accuracy of this metric is poor as you will see from running the code below. For instance, Pullover is classified as a T-Shirt, and a Pant is classified as a Shoe. Overall, the metric is useless and the resultant classification accuracy is poor.

### Complexity

The complexity of KNN for this example is quite high: for each image in Test Set (there are 10000 of them), we compute 60000 metrics (one each for Train image). After populating an array of 60000 metrics, we scan through this array to identify $k$ smallest metrics.

In [0]:
paramk = 11 # parameter k of k-nearest neighbors
numTrainImages = np.shape(trLabels)[0] # so many train images
numTestImages = np.shape(tLabels)[0] # so many test images

arrayKNNLabels = np.array([])
for iTeI in range(1,numTestImages):
  arrayL2Norm = np.array([]) # store distance of a test image from all train images
  for jTrI in range(numTrainImages):  
    l2norm = np.sum(((trImages[jTrI]-tImages[iTeI])/255.0)**2)**(0.5) # distance between two images; 255 is max. pixel value ==> normalization   
    arrayL2Norm = np.append(arrayL2Norm, l2norm)
    
  sIndex = np.argsort(arrayL2Norm) # sorting distance and returning indices that achieves sort
  
  kLabels = trLabels[sIndex[0:paramk]] # choose first k labels  
  (values, counts) = np.unique(kLabels, return_counts=True) # find unique labels and their counts
  arrayKNNLabels = np.append(arrayKNNLabels, values[np.argmax(counts)])
  print(arrayL2Norm[sIndex[0]], kLabels, arrayKNNLabels[-1], tLabels[iTeI])
  
  if arrayKNNLabels[-1] != tLabels[iTeI]:

    plt.figure(1)
    plt.imshow(tImages[iTeI])
    plt.draw()
    
    for i in range(numTrainImages):
      if trLabels[i] == arrayKNNLabels[-1]:
        plt.figure(2)
        plt.imshow(trImages[i])
        plt.draw()
        break
  
    plt.show()

## Classification using L0 Distance

Here we choose to apply L0 distance to determine closeness of Train images to a given Test image. In the code-snippet below, this is achieved in an indirect way (not an elegant solution) but works nevertheless. For every image (Train or Test), we generate a modified image by assigning $1$ to a pixel that is non-zero in the original image and $0$ to a pixel that is zero in the original image. The modified image is a picture with just 1 and 0. To determine closeness of a Test image and a Train image, we compute Euclidean distance metric between their modified images. This metric is proportional to L0 distance on the original copy of the Test and Train images.

Clearly generating a modified image and storing isn't particularly efficient but for purposes of this article it is sufficient. From running the code, I observe that this metric achieves a classification accuracy of **83.8%**, which incidentally isn't too far from what we achieved with a backpropagation network on this dataset.

### Complexity

The complexity of KNN hasn't changed (much) from changing to L0 distance. Therefore, the above complexity discussion still holds true.

### Python-like Implementation

In the snippet below, we implement the algorithm using Python and Numpy.

In [31]:
paramk = 11 # parameter k of k-nearest neighbors
numTrainImages = np.shape(trLabels)[0] # so many train images
numTestImages = np.shape(tLabels)[0] # so many test images

arrayKNNLabels = np.array([])
numErrs = 0
for iTeI in range(0,numTestImages):
  arrayL2Norm = np.array([]) # store distance of a test image from all train images
  
  tmpTImage = np.copy(tImages[iTeI])
  tmpTImage[tmpTImage > 0] = 1
  
  for jTrI in range(numTrainImages):
    tmpTrImage = np.copy(trImages[jTrI])
    tmpTrImage[tmpTrImage>0] = 1
    
    
    l2norm = np.sum(((tmpTrImage-tmpTImage)**2)**(0.5)) # distance between two images; 255 is max. pixel value ==> normalization 
    if jTrI == 0:
      with tf.Session() as sess:
        print(tf.count_nonzero(tmpTrImage-tmpTImage, axis=[0,1]).eval())      
      print(iTeI, jTrI, l2norm)
    arrayL2Norm = np.append(arrayL2Norm, l2norm)
    
  sIndex = np.argsort(arrayL2Norm) # sorting distance and returning indices that achieves sort
  
  kLabels = trLabels[sIndex[0:paramk]] # choose first k labels  
  (values, counts) = np.unique(kLabels, return_counts=True) # find unique labels and their counts
  arrayKNNLabels = np.append(arrayKNNLabels, values[np.argmax(counts)])
   
  if arrayKNNLabels[-1] != tLabels[iTeI]:
    numErrs += 1
    print(numErrs,"/",iTeI)
print("# Classification Errors= ", numErrs, "% accuracy= ", 100.*(numTestImages-numErrs)/numTestImages)

198
0 0 198
first image metric= 198.0
metric= [49938 18339 42774 18094 21346 35541  2688 53939 42778 52468 21342] [32. 33. 34. 35. 40. 40. 41. 41. 44. 45. 46.]
# Classification Errors=  0 % accuracy=  100.0


### Tensorflow-like Implementation

The same algorithm is implemented in a fashion that is best suited for Tensorflow. Here we start-off by defining a detailed graph of the algorithm and then, we run it within a Tensorflow instance of a Session. Tensorflow code took me a little figuring out to put together. If you are new to Tensorflow, I recommend going through this piece of code. If readers experienced with the environment, then feel free to suggest code optimizations.

In [0]:
paramk = 11 # parameter k of K-nearest neighbors

# Defining KNN Graph with L0 Norm
x = tf.placeholder(trImages.dtype, shape=trImages.shape) # all train images, i.e., 60000 x 28 x 28
y = tf.placeholder(tImages.dtype, shape=tImages.shape[1:]) # a test image, 28 x 28

xThresholded = tf.clip_by_value(tf.cast(x, tf.int32), 0, 1) # x is int8 which is not supported in many tf functions, hence typecast
yThresholded = tf.clip_by_value(tf.cast(y, tf.int32), 0, 1) # clip_by_value converts dataset to tensors of 0 and 1, i.e., 1 where tensor is non-zero
computeL0Dist = tf.count_nonzero(xThresholded - yThresholded, axis=[1,2]) # Computing L0 Norm by reducing along axes
findKClosestTrImages = tf.contrib.framework.argsort(computeL0Dist, direction='ASCENDING') # sorting (image) indices in order of ascending metrics, pick first k in the next step
findLabelsKClosestTrImages = tf.gather(trLabels, findKClosestTrImages[0:paramk]) # doing trLabels[findKClosestTrImages[0:paramk]] throws error, hence this workaround
findULabels, findIdex, findCounts = tf.unique_with_counts(findLabelsKClosestTrImages) # examine labels of k closest Train images
findPredictedLabel = tf.gather(findULabels, tf.argmax(findCounts)) # assign label to test image based on most occurring labels among k closest Train images

# Let's run the graph
numErrs = 0
numTestImages = np.shape(tLabels)[0]
with tf.Session() as sess:
  for iTeI in range(0,numTestImages): # iterate each image in test set
    predictedLabel = sess.run([findPredictedLabel], feed_dict={x:trImages, y:tImages[iTeI]})   

    if predictedLabel != tLabels[iTeI]:
      numErrs += 1
      print(numErrs,"/",iTeI)
      print("\t\t", predictedLabel, tLabels[iTeI])
      
print("# Classification Errors= ", numErrs, "% accuracy= ", 100.*(numTestImages-numErrs)/numTestImages)
      

1 / 11
		 [7] 5
2 / 17
		 [6] 4
3 / 21
		 [7] 5
4 / 25
		 [2] 4
5 / 29
		 [6] 3
6 / 40
		 [0] 6
7 / 42
		 [0] 3
8 / 43
		 [9] 7
9 / 49
		 [6] 2
10 / 51
		 [2] 4
11 / 53
		 [2] 8
12 / 66
		 [4] 2
13 / 67
		 [4] 3
14 / 84
		 [7] 5
15 / 89
		 [2] 6
16 / 98
		 [2] 4
17 / 107
		 [7] 9
18 / 117
		 [4] 6
19 / 127
		 [2] 4
20 / 136
		 [4] 2
21 / 139
		 [7] 5
22 / 141
		 [6] 0
23 / 145
		 [2] 6
24 / 147
		 [3] 6
25 / 149
		 [6] 2
26 / 150
		 [3] 4
27 / 151
		 [2] 4
28 / 153
		 [3] 4
29 / 155
		 [2] 0
30 / 172
		 [4] 2
31 / 182
		 [4] 3
32 / 192
		 [3] 1
33 / 197
		 [4] 3
34 / 216
		 [0] 1
35 / 227
		 [6] 2
36 / 236
		 [7] 5
37 / 239
		 [2] 4
38 / 241
		 [4] 6
39 / 244
		 [2] 6
40 / 249
		 [4] 2
41 / 254
		 [9] 5
42 / 258
		 [4] 2
43 / 266
		 [4] 3
44 / 282
		 [2] 6
45 / 283
		 [4] 3
46 / 290
		 [7] 5
47 / 299
		 [3] 8
48 / 312
		 [2] 6
49 / 315
		 [2] 4
50 / 316
		 [7] 9
51 / 324
		 [6] 4
52 / 332
		 [6] 0
53 / 344
		 [0] 6
54 / 354
		 [0] 6
55 / 359
		 [6] 0
56 / 367
		 [4] 2
57 / 369
		 [7] 5

In [15]:
a = np.asarray([[[0,3],[1,2]], [[2,3],[3,2]]])
b = np.asarray([[0,3],[3,3]])
#diff = tf.reduce_sum(tf.count_nonzero(a-b),axis=[])
diff = tf.count_nonzero(a-b, axis=[1,2])
with tf.Session() as sess:
  lNorm = sess.run(diff)
print("lNorm=", lNorm)

lNorm= [2 2]
